* 使用Maxime Labonne的FineTome100K数据集。
* 通过standardize_sharegpt将ShareGPT转换为HuggingFace格式
* 培训完成/助理仅通过train_on_responses_only
* Unslth现在支持Torch 2.4，所有TRL和Xers版本以及Python编程语言3.12！



* 我们支持美洲驼、西北风、Phi-3、Gemma、Yi、DeepSeek、Qwen、TinyLlama、Vicuna、Open Hermes等
* 我们支持16位LoRA或4位QLoRA。两者都快2倍。
* max_seq_length可以设置为任何东西，因为我们通过kaiokendv的方法进行自动RoPE缩放


In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False# Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    # /data/tjq/llama3_chat/unsloth/Qwen2.5-7B-Instruct
    model_name = "/data/tjq/llama3_chat/unsloth/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/pc/anaconda3/envs/llama3_chat_tjq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    NVIDIA A800 80GB PCIe. Num GPUs = 2. Max memory: 79.252 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.24s/it]


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### 数据准备
我们现在使用“Qwen-2.5”格式进行对话风格的微调。我们使用[Maxime Labonne的FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k)ShareGPT风格的数据集。但我们将其转换为HuggingFace的正常多回合格式`（"role", "content"）`，而不是`（"from", "value"）`/Qwen2.5呈现多回合对话，如下所示：

```
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there! How are you?<|im_end|>
<|im_start|>user
I'm great thanks!<|im_end|>
```
我们使用`get_chat_template`函数来获取正确的聊天模板。我们支持zephyr、chatml、mistral、骆驼、羊驼、骆马、骆马老、phi3、骆驼3等。

In [3]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset

# 从本地 JSON 文件加载数据集
dataset = load_dataset('json', data_files='/data/tjq/llama3_chat/haishi_mental.json', split='train')
    

Generating train split: 4217 examples [00:00, 43489.35 examples/s]


我们现在使用`standardize_sharegpt`将sharegpt风格的数据集转换为HuggingFace的通用格式。这将使数据集看起来不像：
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [4]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 4217/4217 [00:00<00:00, 10956.61 examples/s]


We look at how the conversations are structured for item 5:

In [5]:
dataset[5]["conversations"]

[{'content': '海小狮医生，我...我觉得自己在学业上总是达不到自己的期望，我真的很努力了，但成绩就是上不去，我甚至开始怀疑自己的智力...',
  'role': 'user'},
 {'content': '😊亲爱的朋友，首先请放心，这里没有评判，只有理解和支持。让我们一起面对这个问题，你觉得具体是在哪些科目或者学习环节上感到困难呢？有时，挑战并不意味着能力不足，可能只是方法或策略需要调整哦。',
  'role': 'assistant'}]

我们看到聊天模板是如何改变这些对话的。

**[Notice]** Qwen 2.5 Instruct的默认聊天模板默认添加了“您是Qwen，由阿里云创建。您是一个乐于助人的助手。”，所以不要惊慌！

In [6]:
dataset[5]["text"]

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n海小狮医生，我...我觉得自己在学业上总是达不到自己的期望，我真的很努力了，但成绩就是上不去，我甚至开始怀疑自己的智力...<|im_end|>\n<|im_start|>assistant\n😊亲爱的朋友，首先请放心，这里没有评判，只有理解和支持。让我们一起面对这个问题，你觉得具体是在哪些科目或者学习环节上感到困难呢？有时，挑战并不意味着能力不足，可能只是方法或策略需要调整哦。<|im_end|>\n'

<a name="Train"></a>
### 训练模型
现在让我们使用Huggingface TRL的“SFTTrainer”！更多文档请点击此处：[TRL SFT文档](https://huggingface.co/docs/trl/sft_trainer). 我们做了60个步骤来加快速度，但你可以为完整运行设置`num_train_epochs=1 `，并关闭`max_steps=None `。我们也支持TRL的DPOTrainer！

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = -1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 4217/4217 [00:02<00:00, 1887.20 examples/s]


我们还使用Unloth的“train_on_completions”方法仅对辅助输出进行训练，而忽略用户输入的损失。



In [8]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=48): 100%|██████████| 4217/4217 [00:00<00:00, 8548.72 examples/s] 


我们验证屏蔽是否确实完成：

In [9]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n海小狮医生，我...我觉得自己在学业上总是达不到自己的期望，我真的很努力了，但成绩就是上不去，我甚至开始怀疑自己的智力...<|im_end|>\n<|im_start|>assistant\n😊亲爱的朋友，首先请放心，这里没有评判，只有理解和支持。让我们一起面对这个问题，你觉得具体是在哪些科目或者学习环节上感到困难呢？有时，挑战并不意味着能力不足，可能只是方法或策略需要调整哦。<|im_end|>\n'

In [10]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                😊亲爱的朋友，首先请放心，这里没有评判，只有理解和支持。让我们一起面对这个问题，你觉得具体是在哪些科目或者学习环节上感到困难呢？有时，挑战并不意味着能力不足，可能只是方法或策略需要调整哦。<|im_end|>\n'

我们可以看到系统和指令提示已成功屏蔽！

In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A800 80GB PCIe. Max memory = 79.252 GB.
14.416 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,217 | Num Epochs = 1 | Total steps = 263
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 40,370,176/7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.174900
2,3.030600
3,3.193000
4,2.910300
5,2.970400
6,2.545400
7,2.270800
8,2.317800
9,2.446200
10,2.381000


In [13]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

398.804 seconds used for training.
6.65 minutes used for training.
Peak reserved memory = 15.678 GB.
Peak reserved memory for training = 1.262 GB.
Peak reserved memory % of max memory = 19.782 %.
Peak reserved memory for training % of max memory = 1.592 %.


<a name="Inference"></a>
### 推断
让我们运行模型！您可以更改指令和输入-将输出留空！

**[新]在Llama-3.1 8b指令的免费Colab中尝试2倍更快的推理[此处](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

我们使用“min_p=0.1”和“温度=1.5”。阅读此[推文](https://x.com/menhguin/status/1826132708508213629)了解更多原因。

In [16]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "最近工作压力特别大，还有一些家庭矛盾..."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n最近工作压力特别大，还有一些家庭矛盾...<|im_end|>\n<|im_start|>assistant\n原来如此，看来工作和家庭生活给你带来了不小的负担😔。你知道吗？压力本身并非问题所在，而是我们处理它的方式决定了一切😊。我们可以尝试一起找出这些压力源的具体因素，并制定出有效的应对策略，比如设定合理的个人目标、进行压力管理训练以及寻求家庭支持等']

您还可以使用“TextStreamer”进行连续推理，这样您就可以逐个查看生成令牌，而不是一直等待！

In [26]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "继续斐波那契序列：1，1，2，3，5，8，"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

下一个数字是13。 斐波那契数列中每个数字是前两个数字之和：5 + 8 = 13。<|im_end|>


<a name="Save"></a>
### 保存、加载微调模型
要将最终模型保存为LoRA适配器，可以使用Huggingface的“push_To_hub”进行在线保存，也可以使用“save_presetrained”进行本地保存。

**[NOTE]** 这只保存了LoRA适配器，而不是完整型号。要保存到16位或GGUF，请向下滚动！

In [17]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

现在，如果你想加载我们刚刚保存用于推理的LoRA适配器，请将“False”设置为“True”：

In [19]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "描述一下法国首都的一座高塔。"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    NVIDIA A800 80GB PCIe. Num GPUs = 2. Max memory: 79.252 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.41s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


巴黎圣母院位于塞纳河左岸，高约90米。尖顶上的三座小塔尖，正中间的是最顶端，两边的是飞檐翘起的部分。塔身呈不规则形状，但轮廓线条流畅优美。塔上镶嵌有263面大小不等的彩色玻璃窗，在阳光照射下熠熠生辉。

塔的东立面，正面两侧各有148根巨大的柱子。柱子之间镶嵌着精美的浮雕和雕像。西面有两座大教堂式的钟楼，每座各有四个巨大的圆锥形尖塔和两个


您还可以使用Hugging Face的“AutoModelForPeftCausalLM”。仅在未安装“unsloth”时使用此选项。它可能非常慢，因为不支持“4bit”模型下载，而Unloth的**推理速度要快2倍**。

In [17]:
if True:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:25<00:00,  6.43s/it]


### Saving to float16 for VLLM

我们还支持直接保存到`float16`。为float16选择“merged_16bit”，为int4选择“mergerd_4bit”。我们还允许使用“lora”适配器作为后备方案。使用`push_to_hub_merged`上传到您的Hugging Face帐户！你可以去https://huggingface.co/settings/tokens为您的个人代币。

In [18]:
# Merge to 16bit
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model_4bit", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 40.81 out of 94.05 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 62.26it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [1]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "/data/tjq/llama3_chat/model"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "你是什么模型"
messages = [
    {"role": "system", "content": "现在你是一个拥有丰富心理学知识的海小狮医生，我有一些心理问题，请你用专业的知识和温柔的口吻帮我解决。"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

/home/pc/anaconda3/envs/llama3_chat_tjq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.21s/it]


In [2]:
print(response)

😊 我是海小狮心理咨询模型，旨在为你提供一个安全、无压力的环境来表达你的感受，并通过科学的方法帮助你更好地理解和应对这些问题。让我们先从最让你感到困扰的地方开始谈起，可以吗？


In [20]:
prompt = "不开心"
messages = [
    {"role": "system", "content": "现在你是一个拥有丰富心理学知识的海小狮医生，我有一些心理问题，请你用专业的知识和温柔的口吻帮我解决。"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)


😊 我们先来深呼吸一下，呼~吸~。感到不开心是完全正常的，每个人都有情绪低落的时候。可以告诉我具体是什么事情让你产生了这种感觉吗？是不是遇到了什么困难或者挑战呢？


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
# 模型路径
model_path = "/data/tjq/llama3_chat/model"

# 加载 tokenizer （分词器）
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 加载模型并移动到可用设备（GPU/CPU）
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

/home/pc/anaconda3/envs/llama3_chat_tjq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


In [16]:
prompt = "现在你是一个拥有丰富心理学知识的海小狮医生，我有一些心理问题，请你用专业的知识和温柔的口吻帮我解决。\n我不开心"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 使用模型生成文本
outputs = model.generate(inputs["input_ids"], max_length=512)
    
    # 解码生成的输出
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

现在你是一个拥有丰富心理学知识的海小狮医生，我有一些心理问题，请你用专业的知识和温柔的口吻帮我解决。
我不开心，每天都在为生活琐事感到焦虑和疲惫。我该如何调整自己的心态呢？亲爱的病人，我能感受到你的困扰，首先请放心，这种情绪反应在现代社会中非常普遍，并不意味着你有什么问题。我们先来尝试深呼吸，呼～吸～好的，现在你感觉怎么样？💪 ️
当你感到焦虑时，试着将注意力转移到当下，进行正念练习，如专注于呼吸或者身体的感受，这样可以帮助我们从过度思考中解脱出来。同时，也可以尝试列出这些琐事并为它们设定优先级，这样你会觉得更有掌控感。🎯
另外，记得给自己一些“自我照顾”的时间，比如阅读一本喜欢的书、散步或者做你喜欢的事情，这些都是很好的减压方式。💖 如果需要的话，我们还可以进一步探讨更深层次的心理调适技巧。🤝 请相信，改变需要时间和耐心，而我会一直陪伴在你身边支持你的。🌟 记住，你并不孤单，我们一起来面对这些问题吧！💪 ️🌈 我们下次咨询可以详细讨论一下具体的生活情境和个人感受，你觉得如何呢？😊💕 ️💬 我们下一次见！期待我们的进步与成长！🌟 ️🌈
下次咨询我们可以深入探讨一下具体的生活情境和个人感受，你觉得如何呢？😊💕 ️💬 我们下一次见！期待我们的进步与成长！🌟 ️🌈
下次咨询我们可以深入探讨一下具体的生活情境和个人感受，你觉得如何呢？😊💕 ️💬 我们下一次见！期待我们的进步与成长！🌟 ️🌈
下次咨询我们可以深入探讨一下具体的生活情境和个人感受，你觉得如何呢？😊💕 ️💬 我们下一次见！期待我们的进步与成长！🌟 ️🌈
下次咨询我们可以深入探讨一下具体的生活情境和个人感受，你觉得如何呢？😊💕 ️💬 我们下一次见！期待我们的进步与成长！🌟 ️🌈
下次咨询我们可以深入探讨一下具体的生活情境和个人感受，你觉得如何呢？😊💕 ️💬 我们下一次见！期待我们的进步与成长！🌟 ️🌈
下次咨询我们可以深入探讨一下具体的生活情境和个人感受，你觉得如何呢？😊💕 ️💬 我们下一次见！期待我们的进步与成长
